In [ ]:
#first i need data from a previous run
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, metrics, Model
from tensorflow.keras.callbacks import Callback
from scipy.stats import truncnorm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import normalize
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import numpy as np
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import h5py
import datetime
from scipy.optimize import curve_fit
import math
from scipy import asarray as ar,exp
import pylab as plb
import tqdm
from tqdm import tqdm
import time
import progressbar

In [ ]:
dataset = np.load("/home/roberts/O-PPAC/recomplied_dataset.npy")
all_data = dataset[ : , :132]
all_labels = dataset[: , -2:]

print(all_data.shape)
print(all_labels.shape)

In [ ]:
all_data = normalize(all_data,axis = 1)
print(np.max(all_data))

In [ ]:
#separate the test data as done above
x1_data_all = (all_data[: , :33])
x2_data_all = (all_data[:, 33:66])
y1_data_all = (all_data[:, 66:99])
y2_data_all = (all_data[:, 99:132])

#labels kept so they have the same indices
x_labels_all = all_labels[ : , 0 ]
y_labels_all = all_labels[ : , 1 ]

print(x1_data_all.shape)
print(x_labels_all.shape)

In [ ]:
#gaussian fit function
def gaussianpdf(x, k, sigma, mu):
    return k*np.exp(-(x-mu)**2/(2*sigma**2))

In [ ]:
#function to (hopefully) fit gaussians (and thus, centroids which is what we want) onto all data sets
def centroiddn(x, y):
    try:
        if np.max(y) > 20:
            kvalue = np.max(y)+(np.max(y)/2)
        elif np.max(y) == 0:
            kvalue = 1
        else:
            kvalue = np.max(y)
        popt, _ = curve_fit(gaussianpdf, x, y, bounds=(0, [kvalue,10., 33.]))
        return popt[2]
    except RuntimeError:
        return -5555

In [ ]:
def fit_peaks(x_data, y_data):
    fit_peaks = []
    for i in tqdm(range(y_data.shape[0])):
        fit_peaks.append(centroiddn(x_data, y_data[i,:]))
    return np.array(fit_peaks)

In [ ]:
def find_peak_error(arr1data, arr2data):
    error_indices1 = np.where(arr1data == -5555 )
    error_indices2 = np.where(arr2data == -5555 )
        
    error_indices = np.unique(np.concatenate([error_indices1, error_indices2], axis = 1))
        
    return error_indices

In [ ]:
def remove_peak_error(arr1data, arr2data, removearr):
    
    arr1data_df = pd.DataFrame(arr1data)
    arr2data_df = pd.DataFrame(arr2data)
        
    arr1data_df_fixed = arr1data_df.drop(arr1data_df.index[removearr])
    arr2data_df_fixed = arr2data_df.drop(arr2data_df.index[removearr])
        
    arr1data_fixed = arr1data_df_fixed.to_numpy()
    arr2data_fixed = arr2data_df_fixed.to_numpy()
    
    arr1data = np.reshape(arr1data_fixed, -1)
    arr2data = np.reshape(arr2data_fixed, -1)
    
    return arr1data, arr2data

In [ ]:
def remove_label_indices(labels, indices):
    labels = pd.DataFrame(labels)
    labels_fixed = labels.drop(labels.index[indices])
    labels = labels_fixed.to_numpy()
    return np.reshape(labels, -1)

In [ ]:
#conversions from diode number to millimeters and vice versa
#might need to be updated if Yassid sends a better method of converting
def diodenum_to_mm(arr1data):
    return (arr1data /33)*100 - 48

def mm_to_diodenum(arr1data):
    return ((arr1data + 48)/100)*33

In [ ]:
def average(arr1data, arr2data):
    avg = (arr1data + arr2data)/2
    return avg

In [ ]:
def peak_recompile(arr1, arr2, arrerror):
    arr1, arr2 = remove_peak_error(arr1, arr2, arrerror)
    avg_peaks = average(arr1, arr2)
    mm_peaks = diodenum_to_mm(avg_peaks)
    return avg_peaks, mm_peaks

In [ ]:
rangemin = 0
rangemax = 200000

ranges = (rangemin, rangemax)
range_size = rangemax - rangemin

x1_data = x1_data_all[rangemin:rangemax]
x2_data = x2_data_all[rangemin:rangemax]
y1_data = y1_data_all[rangemin:rangemax]
y2_data = y2_data_all[rangemin:rangemax]

x_labels = x_labels_all[rangemin:rangemax]
y_labels = y_labels_all[rangemin:rangemax]

In [ ]:
x1_peaks = fit_peaks(np.arange(0,33,1), x1_data)

In [ ]:
np.save("x1_peaks_file_2", x1_peaks)

In [ ]:
x2_peaks = fit_peaks(np.arange(0,33,1), x2_data)

In [ ]:
np.save("x2_peaks_file_2", x2_peaks)

In [ ]:
y1_peaks = fit_peaks(np.arange(0,33,1), y1_data)

In [ ]:
np.save("y1_peaks_file_2", y1_peaks)

In [ ]:
y2_peaks = fit_peaks(np.arange(0,33,1), y2_data)

In [ ]:
np.save("y2_peaks_file_2", y2_peaks)

In [ ]:
#importing data so I'm not running over the other kernels
x1_peaks = np.load("x1_peaks_file_2.npy")
x2_peaks = np.load("x2_peaks_file_2.npy")
y1_peaks = np.load("y1_peaks_file_2.npy")
y2_peaks = np.load("y2_peaks_file_2.npy")

print(len(x1_peaks))
print(len(x2_peaks))
print(len(y1_peaks))
print(len(y2_peaks))

print(x1_peaks.shape)

In [ ]:
plt.plot(x1_data[46277])
plt.axvline(x = x1_peaks[46277])
plt.show()

In [ ]:
x_errors = find_peak_error(x1_peaks, x2_peaks)
y_errors = find_peak_error(y1_peaks, y2_peaks)

all_error_indices = np.unique(np.concatenate([x_errors, y_errors]))

all_error_indices = np.array(all_error_indices)
np.save("all_error_indices", all_error_indices)
print(all_error_indices)

In [ ]:
x_peaks, x_peaks_mm = peak_recompile(x1_peaks, x2_peaks, all_error_indices)

In [ ]:
y_peaks, y_peaks_mm = peak_recompile(y1_peaks, y2_peaks, all_error_indices)

In [ ]:
x_labels = remove_label_indices(x_labels, all_error_indices)
y_labels = remove_label_indices(y_labels, all_error_indices)

print(x_labels.shape)
print(x_peaks_mm.shape)
print(y_labels.shape)
print(y_peaks_mm.shape)

In [ ]:
#graph showing the accuracy of gaussian fit by comparing predicted and true x values
#should be as close to the diagonal as possible
a = plt.axes(aspect='equal')

plt.scatter(x_labels, x_pea, s=6, marker = "x")
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [-50, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims, linewidth = .5)
plt.title("Gaussian Fit O-PPAC X Predictions vs True X Values (events %5.0f - %5.0f)" %tuple(ranges))
plt.show()

In [ ]:
#to find the locations of largest error within my predicted x values
x_error = np.abs(x_peaks_mm - x_labels)

print(x_error.shape)

problem_indices_x = [idx for (idx, err) in enumerate(x_error) if err >= 10]
print(len(problem_indices_x))

In [ ]:
k= 46277

print(x_peaks_mm[k])
print(mm_to_diodenum(x_peaks_mm[k]))
print(x_labels[k])

In [ ]:
i = 5 #choose the index of the error (above)
true_x = mm_to_diodenum(x_labels[problem_indices_x[i]])

plt.title('Problem Location Example (index = %5.0f)' %(problem_indices_x[i]+rangemin))
plt.plot(x1_data_all[problem_indices_x[i] + rangemin], linewidth = 2, label = "data")
plt.axvline(x = x_peaks[problem_indices_x[i]], color = "red", linestyle=":",label = "fit peak = %5.1f" %x_peaks[problem_indices_x[i]])
plt.axvline(x = x1_peaks[problem_indices_x[i]], color = "black", linewidth = 1, linestyle=":", label = "x1 fit peak = %5.1f" % x1_peaks[problem_indices_x[i]])
plt.axvline(x = x2_peaks[problem_indices_x[i]], color = "magenta", linewidth = 1, linestyle=":", label = "x2 fit peak = %5.1f" % x2_peaks[problem_indices_x[i]])
plt.axvline(x = true_x, color = "green", linestyle=":",label = "true peak = %5.1f" %true_x)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

In [ ]:
#graph showing the accuracy of gaussian fit by comparing predicted and true y values
#should be as close to the diagonal as possible
a = plt.axes(aspect='equal')

plt.scatter(y_labels, y_peaks_mm, s=6, marker = "x")
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [-50, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims, linewidth = .5)
plt.title("Gaussian Fit O-PPAC Y Predictions vs True Y Values (events %5.0f - %5.0f)" %tuple(ranges))
plt.show()

In [ ]:
#to find the locations of largest error within my predicted x values
y_error = np.abs(y_peaks_mm - y_labels)

print(y_error.shape)

problem_indices_y = [idx for (idx, err) in enumerate(y_error) if err >= 10]
print(problem_indices_y)

In [ ]:
#graph locations of large y error
i = 0 #choose the index of the error (above)

plt.title('Problem Location Example')
plt.plot(y1_data_all[5125 + rangemin])
plt.axvline(x = y1_peaks[problem_indices_y[i]], color = "red")
plt.axvline(x = y_labels[problem_indices_y[i]], color = "green")
plt.show()

In [ ]:
#basic gaussian fit stuff; needs x and y inputs for all data points; index is both for the y data and for lower code
index = 138450
x = np.arange(0,33,1)
y = x1_data[index,:]

print(y)
x_labels_index = range_size - (rangemax - index)

#the true location of the peak as given by the label for that event
x_peak_true_value = ((x_labels[x_labels_index]+48)/100)*33
x_peak_fit_value = x_peaks[x_labels_index]

In [ ]:
#curve fit code; bounds might be able to be played with still to improve model (specifically sigma in index 1)
#trying new stuff to better balance k based on the input data
if np.max(y) > 20:
    kvalue = np.max(y)+(np.max(y)/2)
elif np.max(y) == 0:
    kvalue = 1
else:
    kvalue = np.max(y)

popt, _ = curve_fit(gaussianpdf, x, y, bounds=(0, [kvalue,10., 33.]))
print(popt[2])
print(x_peak_true_value)
print(x_peak_fit_value)

In [ ]:
#plotting gaussian to make sure it looks logically correct
plt.plot(x, gaussianpdf(x, *popt), 'g--', label='fit: k=%5.3f, sigma=%5.3f, mu=%5.3f' % tuple(popt))
plt.plot(x1_data[index,:], label = "data")
plt.axvline(x = x_peak_fit_value, color = "black", linewidth=.4, label='Pred Peak= %5.1f' %x_peak_fit_value)
plt.axvline(x = x_peak_true_value, color = "magenta", linewidth=1, label='True Peak= %5.1f' % x_peak_true_value)
plt.axvline(x = popt[2], color = "red", linewidth=.4, linestyle='dashdot', label='Fit Peak= %5.1f' %popt[2])
plt.title('Event Number %5.0f' %index)
plt.xlabel('Diode Number')
plt.ylabel('Number of Detected Photons')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()